<a href="https://colab.research.google.com/github/abuarafatanik-cyber/Assignment-3/blob/main/Assignment%233.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. Install necessary library
!pip install pdfplumber

import pdfplumber
import pandas as pd
import re
from google.colab import files
from typing import List, Dict

# List of the files you uploaded
PDF_FILES = [
    "September 2020.pdf",
    "October 2020.pdf",
    "November 2020.pdf",
    "December 2020.pdf"
]

# 2. Upload your files
print("Please upload the four PDF files when prompted:")
uploaded = files.upload()

# 3. Define the extraction function

def extract_transactions_from_pdf(file_name: str) -> List[Dict]:
    """
    Extracts transaction data from a single CIBC statement PDF using pdfplumber's
    table extraction functionality (template extraction).
    """
    transactions_list = []
    statement_period = "N/A"

    print(f"\n--- Processing {file_name} ---")

    try:
        with pdfplumber.open(file_name) as pdf:
            # Get the statement period from the first page for context
            first_page_text = pdf.pages[0].extract_text()
            period_match = re.search(r'statement period\s+(.*?)\.', first_page_text, re.IGNORECASE)
            if period_match:
                statement_period = period_match.group(1).strip()

            # Iterate through all pages
            for page in pdf.pages:
                # Use default table settings which are usually effective for bank statements
                tables = page.extract_tables()

                for table in tables:
                    # Look for tables that have the structure of transactions.
                    # Transaction tables typically have 5 or more columns.
                    if table and len(table[0]) >= 5:

                        # Find the header row (it usually contains 'Date' and 'Description')
                        header_index = -1
                        for i, row in enumerate(table):
                            if row and any("DATE" in str(cell).upper() and "DESCRIPTION" in str(cell).upper() for cell in row):
                                header_index = i
                                break

                        if header_index != -1:
                            # Heuristic: Assume columns are in order: Txn Date, Post Date, Description, Reference, Amount
                            # (Some columns might be merged or missing, so we focus on the first 5-6 non-empty values)

                            # Start processing data from the row after the header
                            for row in table[header_index + 1:]:

                                # Filter out empty or non-transaction rows (like page footers/headers)
                                clean_row = [str(cell).strip() for cell in row if str(cell).strip()]

                                # A typical transaction row should have at least 4 key pieces of info
                                if len(clean_row) >= 4 and re.match(r'^\w{3}\s\d{1,2}$', clean_row[0]):

                                    # Clean up the amount: remove commas, convert 'CR' (Credit) to negative.
                                    amount_str = clean_row[-1].replace('$', '').replace(',', '').strip()
                                    is_credit = False
                                    if amount_str.endswith('CR'):
                                        amount_str = amount_str.replace('CR', '').strip()
                                        is_credit = True

                                    try:
                                        amount = float(amount_str)
                                        if is_credit:
                                            amount *= -1 # Treat credits as negative amounts
                                    except ValueError:
                                        # Skip rows where amount can't be parsed (e.g., summary rows)
                                        continue

                                    # The columns are typically [Date, Post Date, Description, Reference, Amount]
                                    transaction = {
                                        "Statement_Period": statement_period,
                                        "Transaction Date": clean_row[0],
                                        "Transaction Post Date": clean_row[1],
                                        # Combine the rest of the available non-date/amount columns into the description
                                        "Transaction Description": " ".join(clean_row[2:-1]),
                                        "Spend Category": "N/A (Must be manually assigned)",
                                        "Amount": amount
                                    }

                                    transactions_list.append(transaction)

            print(f"Successfully extracted {len(transactions_list)} transactions.")

    except FileNotFoundError:
        print(f"Error: File {file_name} not found. Ensure it was uploaded correctly.")
    except Exception as e:
        print(f"An error occurred while processing {file_name}: {e}")

    return transactions_list


# 4. Main execution and structuring into an array
all_transactions_data = []

for file_name in PDF_FILES:
    if file_name in uploaded:
        all_transactions_data.extend(extract_transactions_from_pdf(file_name))
    else:
        print(f"\nSkipping {file_name}: File was not uploaded.")

# 5. Print the final array data structure (List of Dictionaries)
print("\n" + "="*50)
print("FINAL EXTRACTED TRANSACTION DATA (Array Structure)")
print("="*50)
print(all_transactions_data)

# Optional: Display as a clean DataFrame
if all_transactions_data:
    print("\n" + "="*50)
    print("OPTIONAL: DISPLAY AS PANDAS DATAFRAME FOR EASIER VIEWING")
    print("="*50)
    df = pd.DataFrame(all_transactions_data)
    print(df.to_markdown(index=False))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 35.9 MB/s eta 0:00:00
Please upload the four PDF files when prompted:


Saving December 2020.pdf to December 2020.pdf
Saving November 2020.pdf to November 2020.pdf
Saving October 2020.pdf to October 2020.pdf
Saving September 2020.pdf to September 2020.pdf

--- Processing September 2020.pdf ---
Successfully extracted 0 transactions.

--- Processing October 2020.pdf ---
Successfully extracted 0 transactions.

--- Processing November 2020.pdf ---
Successfully extracted 0 transactions.

--- Processing December 2020.pdf ---
Successfully extracted 0 transactions.

FINAL EXTRACTED TRANSACTION DATA (Array Structure)
[]


In [2]:
# ----------------------------------------------------------------------
# Interactive Spend Report for Google Colab
# This script uses pandas and plotly to replicate the monthly spend
# visualization within a Python environment (Google Colab).
# ----------------------------------------------------------------------

# 1. Install necessary libraries if not already present in the Colab runtime
# Note: plotly is usually pre-installed in Colab.
# !pip install pandas plotly

import pandas as pd
import plotly.express as px
from ipywidgets import interact, Dropdown, Layout
from IPython.display import display, HTML

# --- MOCK TRANSACTION DATA ---
# This mirrors the data structure extracted from your PDF files.
MOCK_TRANSACTIONS = [
    # September 2020 (Aug 14 - Sep 13)
    {'month': 'September 2020', 'date': 'Sep 10', 'description': 'LOBLAWS GROCERY STORE', 'amount': 85.12, 'category': 'Groceries'},
    {'month': 'September 2020', 'date': 'Sep 11', 'description': 'AMAZON PRIME SUBSCRIPTION', 'amount': 12.99, 'category': 'Subscriptions'},
    {'month': 'September 2020', 'date': 'Sep 12', 'description': 'COFFEE SHOP TIM HORTONS', 'amount': 4.50, 'category': 'Food & Dining'},
    {'month': 'September 2020', 'date': 'Sep 13', 'description': 'CIBC PAYMENT - THANK YOU', 'amount': -150.18, 'category': 'Payment'},

    # October 2020 (Sep 14 - Oct 13)
    {'month': 'October 2020', 'date': 'Sep 15', 'description': 'LCBO LIQUOR STORE', 'amount': 35.80, 'category': 'Entertainment'},
    {'month': 'October 2020', 'date': 'Sep 25', 'description': 'BELL CANADA INTERNET', 'amount': 95.00, 'category': 'Utilities'},
    {'month': 'October 2020', 'date': 'Oct 01', 'description': 'WALMART SUPERCENTER', 'amount': 120.45, 'category': 'Shopping'},
    {'month': 'October 2020', 'date': 'Oct 05', 'description': 'PETSMART SUPPLIES', 'amount': 45.99, 'category': 'Pets'},
    {'month': 'October 2020', 'date': 'Oct 10', 'description': 'UBER EATS DELIVERY', 'amount': 32.70, 'category': 'Food & Dining'},
    {'month': 'October 2020', 'date': 'Oct 13', 'description': 'SHELL GAS STATION', 'amount': 62.10, 'category': 'Automotive'},

    # November 2020 (Oct 14 - Nov 13)
    {'month': 'November 2020', 'date': 'Oct 20', 'description': 'METRO GROCERY STORE', 'amount': 155.60, 'category': 'Groceries'},
    {'month': 'November 2020', 'date': 'Oct 25', 'description': 'NETFLIX SUBSCRIPTION', 'amount': 16.99, 'category': 'Subscriptions'},
    {'month': 'November 2020', 'date': 'Nov 01', 'description': 'AMAZON.CA PURCHASE', 'amount': 210.50, 'category': 'Shopping'},
    {'month': 'November 2020', 'date': 'Nov 05', 'description': 'DRY CLEANERS SERVICE', 'amount': 22.00, 'category': 'Personal Care'},
    {'month': 'November 2020', 'date': 'Nov 12', 'description': 'CINEPLEX MOVIES', 'amount': 28.00, 'category': 'Entertainment'},

    # December 2020 (Nov 14 - Dec 13)
    {'month': 'December 2020', 'date': 'Nov 15', 'description': 'ROGERS CELL PHONE BILL', 'amount': 75.00, 'category': 'Utilities'},
    {'month': 'December 2020', 'date': 'Nov 20', 'description': 'APPLE STORE INC PURCHASE', 'amount': 350.99, 'category': 'Shopping'},
    {'month': 'December 2020', 'date': 'Dec 05', 'description': 'FOOD BASICS GROCERY', 'amount': 98.30, 'category': 'Groceries'},
    {'month': 'December 2020', 'date': 'Dec 10', 'description': 'RESTAURANT TAKE OUT', 'amount': 45.00, 'category': 'Food & Dining'},
    {'month': 'December 2020', 'date': 'Dec 12', 'description': 'CAR WASH DETAIL', 'amount': 30.00, 'category': 'Automotive'},
]

# Create a master DataFrame
df_master = pd.DataFrame(MOCK_TRANSACTIONS)
df_master['amount'] = df_master['amount'].round(2) # Ensure amounts are clean
MONTHS = sorted(df_master['month'].unique())
LATEST_MONTH = MONTHS[-1]

# 2. Function to generate the interactive report
def generate_spend_report(selected_month):
    """Generates and displays the category pie chart and transaction table."""
    print(f"--- Spend Report for {selected_month} ---")

    # Filter for the selected month and exclude credits/payments (amount > 0)
    df_monthly_spending = df_master[
        (df_master['month'] == selected_month) &
        (df_master['amount'] > 0) &
        (df_master['category'] != 'Payment')
    ]

    # Aggregate by Category (for Pie Chart)
    if df_monthly_spending.empty:
        print("\nNo purchase transactions found for this period to visualize.")
        return

    df_category_spend = df_monthly_spending.groupby('category')['amount'].sum().reset_index()
    df_category_spend = df_category_spend.sort_values(by='amount', ascending=False)

    # Calculate Summary Stats
    total_spend = df_monthly_spending['amount'].sum()
    num_transactions = df_monthly_spending.shape[0]

    # --- DISPLAY SUMMARY STATS ---
    print(f"\nTotal Purchases: ${total_spend:.2f}")
    print(f"Total Transactions: {num_transactions}")
    print("-" * 30)

    # --- 1. CATEGORY BREAKDOWN (PIE CHART) ---
    print("\n[1] Category Breakdown Pie Chart:")
    fig_pie = px.pie(
        df_category_spend,
        values='amount',
        names='category',
        title=f'Spending Breakdown in {selected_month}',
        color_discrete_sequence=px.colors.qualitative.Bold,
        hole=0.3
    )
    fig_pie.update_traces(textposition='inside', textinfo='percent+label')
    fig_pie.show()

    # --- 2. DETAILED TRANSACTION TABLE ---
    print("\n[2] Detailed Transactions Table:")
    # Only show relevant columns for the table output
    df_table = df_monthly_spending[['date', 'description', 'category', 'amount']]

    # Format the table for better display in Colab
    display(HTML(f"<h3>Detailed Transactions for {selected_month}</h3>"))
    display(df_table.style.set_properties(**{'font-size': '10pt', 'border': '1px solid black'})
                          .format({'amount': '${:.2f}'}))


# 3. Monthly Comparison Chart (Static - does not depend on dropdown)
def plot_monthly_comparison(df):
    """Plots the total monthly spending across all available months (Bar Chart)."""
    # Filter for positive spending (purchases)
    df_purchases = df[df['amount'] > 0]

    # Aggregate total spending per month
    df_monthly_total = df_purchases.groupby('month')['amount'].sum().reset_index()

    print("\n" + "="*50)
    print("Monthly Spending Comparison (Bar Chart)")
    print("="*50)

    fig_bar = px.bar(
        df_monthly_total,
        x='month',
        y='amount',
        title='Total Spending Across All Statement Periods',
        labels={'amount': 'Total Spend ($)', 'month': 'Statement Month'},
        color='amount',
        color_continuous_scale=px.colors.sequential.Sunsetdark
    )
    fig_bar.update_layout(xaxis={'categoryorder':'array', 'categoryarray':MONTHS})
    fig_bar.update_traces(texttemplate='$%{y:.2f}', textposition='outside')
    fig_bar.show()

# 4. Execute the interactive elements
# Plot the overall comparison first
plot_monthly_comparison(df_master)

# Create the Dropdown widget
month_selector = Dropdown(
    options=MONTHS,
    value=LATEST_MONTH,
    description='Select Month:',
    style={'description_width': 'initial'},
    layout=Layout(width='300px')
)

# Use the interact function to link the dropdown to the report generator
display(HTML("<h2><center>Interactive Monthly Spend Analyzer</center></h2>"))
interact(generate_spend_report, selected_month=month_selector)



Monthly Spending Comparison (Bar Chart)


interactive(children=(Dropdown(description='Select Month:', index=3, layout=Layout(width='300px'), options=('D…

<function __main__.generate_spend_report(selected_month)>

In [3]:
# ----------------------------------------------------------------------
# Interactive Spend Report for Google Colab
# This script uses pandas and plotly to replicate the monthly spend
# visualization within a Python environment (Google Colab).
# ----------------------------------------------------------------------

# 1. Install necessary libraries if not already present in the Colab runtime
# Note: plotly is usually pre-installed in Colab.
# We need scikit-learn for the machine learning model.
!pip install scikit-learn

import pandas as pd
import plotly.express as px
from ipywidgets import interact, Dropdown, Layout
from IPython.display import display, HTML

# ML imports
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# --- MOCK TRANSACTION DATA ---
# This mirrors the data structure extracted from your PDF files.
MOCK_TRANSACTIONS = [
    # September 2020 (Aug 14 - Sep 13)
    {'month': 'September 2020', 'date': 'Sep 10', 'description': 'LOBLAWS GROCERY STORE', 'amount': 85.12, 'category': 'Groceries'},
    {'month': 'September 2020', 'date': 'Sep 11', 'description': 'AMAZON PRIME SUBSCRIPTION', 'amount': 12.99, 'category': 'Subscriptions'},
    {'month': 'September 2020', 'date': 'Sep 12', 'description': 'COFFEE SHOP TIM HORTONS', 'amount': 4.50, 'category': 'Food & Dining'},
    {'month': 'September 2020', 'date': 'Sep 13', 'description': 'CIBC PAYMENT - THANK YOU', 'amount': -150.18, 'category': 'Payment'},

    # October 2020 (Sep 14 - Oct 13)
    {'month': 'October 2020', 'date': 'Sep 15', 'description': 'LCBO LIQUOR STORE', 'amount': 35.80, 'category': 'Entertainment'},
    {'month': 'October 2020', 'date': 'Sep 25', 'description': 'BELL CANADA INTERNET', 'amount': 95.00, 'category': 'Utilities'},
    {'month': 'October 2020', 'date': 'Oct 01', 'description': 'WALMART SUPERCENTER', 'amount': 120.45, 'category': 'Shopping'},
    {'month': 'October 2020', 'date': 'Oct 05', 'description': 'PETSMART SUPPLIES', 'amount': 45.99, 'category': 'Pets'},
    {'month': 'October 2020', 'date': 'Oct 10', 'description': 'UBER EATS DELIVERY', 'amount': 32.70, 'category': 'Food & Dining'},
    {'month': 'October 2020', 'date': 'Oct 13', 'description': 'SHELL GAS STATION', 'amount': 62.10, 'category': 'Automotive'},

    # November 2020 (Oct 14 - Nov 13)
    {'month': 'November 2020', 'date': 'Oct 20', 'description': 'METRO GROCERY STORE', 'amount': 155.60, 'category': 'Groceries'},
    {'month': 'November 2020', 'date': 'Oct 25', 'description': 'NETFLIX SUBSCRIPTION', 'amount': 16.99, 'category': 'Subscriptions'},
    {'month': 'November 2020', 'date': 'Nov 01', 'description': 'AMAZON.CA PURCHASE', 'amount': 210.50, 'category': 'Shopping'},
    {'month': 'November 2020', 'date': 'Nov 05', 'description': 'DRY CLEANERS SERVICE', 'amount': 22.00, 'category': 'Personal Care'},
    {'month': 'November 2020', 'date': 'Nov 12', 'description': 'CINEPLEX MOVIES', 'amount': 28.00, 'category': 'Entertainment'},

    # December 2020 (Nov 14 - Dec 13)
    {'month': 'December 2020', 'date': 'Nov 15', 'description': 'ROGERS CELL PHONE BILL', 'amount': 75.00, 'category': 'Utilities'},
    {'month': 'December 2020', 'date': 'Nov 20', 'description': 'APPLE STORE INC PURCHASE', 'amount': 350.99, 'category': 'Shopping'},
    {'month': 'December 2020', 'date': 'Dec 05', 'description': 'FOOD BASICS GROCERY', 'amount': 98.30, 'category': 'Groceries'},
    {'month': 'December 2020', 'date': 'Dec 10', 'description': 'RESTAURANT TAKE OUT', 'amount': 45.00, 'category': 'Food & Dining'},
    {'month': 'December 2020', 'date': 'Dec 12', 'description': 'CAR WASH DETAIL', 'amount': 30.00, 'category': 'Automotive'},
]

# Create a master DataFrame
df_master = pd.DataFrame(MOCK_TRANSACTIONS)
df_master['amount'] = df_master['amount'].round(2) # Ensure amounts are clean
MONTHS = sorted(df_master['month'].unique())
LATEST_MONTH = MONTHS[-1]

# 2. Function to generate the interactive report
def generate_spend_report(selected_month):
    """Generates and displays the category pie chart and transaction table."""
    print(f"--- Spend Report for {selected_month} ---")

    # Filter for the selected month and exclude credits/payments (amount > 0)
    df_monthly_spending = df_master[
        (df_master['month'] == selected_month) &
        (df_master['amount'] > 0) &
        (df_master['category'] != 'Payment')
    ]

    # Aggregate by Category (for Pie Chart)
    if df_monthly_spending.empty:
        print("\nNo purchase transactions found for this period to visualize.")
        return

    df_category_spend = df_monthly_spending.groupby('category')['amount'].sum().reset_index()
    df_category_spend = df_category_spend.sort_values(by='amount', ascending=False)

    # Calculate Summary Stats
    total_spend = df_monthly_spending['amount'].sum()
    num_transactions = df_monthly_spending.shape[0]

    # --- DISPLAY SUMMARY STATS ---
    print(f"\nTotal Purchases: ${total_spend:.2f}")
    print(f"Total Transactions: {num_transactions}")
    print("-" * 30)

    # --- 1. CATEGORY BREAKDOWN (PIE CHART) ---
    print("\n[1] Category Breakdown Pie Chart:")
    fig_pie = px.pie(
        df_category_spend,
        values='amount',
        names='category',
        title=f'Spending Breakdown in {selected_month}',
        color_discrete_sequence=px.colors.qualitative.Bold,
        hole=0.3
    )
    fig_pie.update_traces(textposition='inside', textinfo='percent+label')
    fig_pie.show()

    # --- 2. DETAILED TRANSACTION TABLE ---
    print("\n[2] Detailed Transactions Table:")
    # Only show relevant columns for the table output
    df_table = df_monthly_spending[['date', 'description', 'category', 'amount']]

    # Format the table for better display in Colab
    display(HTML(f"<h3>Detailed Transactions for {selected_month}</h3>"))
    display(df_table.style.set_properties(**{'font-size': '10pt', 'border': '1px solid black'})
                          .format({'amount': '${:.2f}'}))


# 3. Monthly Comparison Chart (Static - does not depend on dropdown)
def plot_monthly_comparison(df):
    """Plots the total monthly spending across all available months (Bar Chart)."""
    # Filter for positive spending (purchases)
    df_purchases = df[df['amount'] > 0]

    # Aggregate total spending per month
    df_monthly_total = df_purchases.groupby('month')['amount'].sum().reset_index()

    print("\n" + "="*50)
    print("Monthly Spending Comparison (Bar Chart)")
    print("="*50)

    fig_bar = px.bar(
        df_monthly_total,
        x='month',
        y='amount',
        title='Total Spending Across All Statement Periods',
        labels={'amount': 'Total Spend ($)', 'month': 'Statement Month'},
        color='amount',
        color_continuous_scale=px.colors.sequential.Sunsetdark
    )
    fig_bar.update_layout(xaxis={'categoryorder':'array', 'categoryarray':MONTHS})
    fig_bar.update_traces(texttemplate='$%{y:.2f}', textposition='outside')
    fig_bar.show()

# 4. Regression Machine Learning Model (Predict Total Spend)
def train_spend_predictor(df):
    """
    Trains a simple Linear Regression model to predict the transaction amount
    based on month and category.
    """
    print("\n" + "="*50)
    print("REGRESSION MODEL: PREDICTING TRANSACTION AMOUNT")
    print("="*50)

    # 4.1 Data Preparation: Filter for purchases only
    df_purchases = df[(df['amount'] > 0) & (df['category'] != 'Payment')].copy()

    if df_purchases.shape[0] < 5:
        print("Not enough data points (purchases) to train a meaningful model.")
        return

    # Define features (X) and target (y)
    X = df_purchases[['month', 'category']]
    y = df_purchases['amount']

    # 4.2 Feature Engineering (One-Hot Encoding for Categorical Data)
    # This transforms categories and months into numerical features the model can use.
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(handle_unknown='ignore'), ['month', 'category'])
        ],
        remainder='passthrough'
    )

    # 4.3 Split Data (70% Train, 30% Test)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42
    )

    # 4.4 Apply Preprocessing and Train Model
    # Apply the encoder to the data
    X_train_processed = preprocessor.fit_transform(X_train)
    X_test_processed = preprocessor.transform(X_test)

    # Train the Linear Regression Model (Simple model helps avoid overfitting on small data)
    model = LinearRegression()
    model.fit(X_train_processed, y_train)

    # 4.5 Prediction and Evaluation
    y_pred = model.predict(X_test_processed)

    # Evaluate performance
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f"Model Trained on {X_train.shape[0]} samples, Tested on {X_test.shape[0]} samples.")
    print(f"\n--- Model Performance ---")
    print(f"Mean Absolute Error (MAE): ${mae:.2f} (Average prediction error)")
    print(f"R-squared (R²): {r2:.2f} (Closer to 1.0 is better)")
    print("-------------------------\n")

    # 4.6 Display Sample Predictions
    predictions_df = pd.DataFrame({'Actual Amount': y_test, 'Predicted Amount': y_pred.round(2)})
    predictions_df['Difference'] = predictions_df['Actual Amount'] - predictions_df['Predicted Amount']

    print("Sample Test Predictions:")
    display(predictions_df.style.format({'Actual Amount': '${:.2f}', 'Predicted Amount': '${:.2f}', 'Difference': '{:.2f}'}))

# 5. Execute the interactive elements
# Plot the overall comparison first
plot_monthly_comparison(df_master)

# Train and evaluate the predictive model
train_spend_predictor(df_master)

# Create the Dropdown widget
month_selector = Dropdown(
    options=MONTHS,
    value=LATEST_MONTH,
    description='Select Month:',
    style={'description_width': 'initial'},
    layout=Layout(width='300px')
)

# Use the interact function to link the dropdown to the report generator
display(HTML("<h2><center>Interactive Monthly Spend Analyzer</center></h2>"))
interact(generate_spend_report, selected_month=month_selector)



Monthly Spending Comparison (Bar Chart)



REGRESSION MODEL: PREDICTING TRANSACTION AMOUNT
Model Trained on 13 samples, Tested on 6 samples.

--- Model Performance ---
Mean Absolute Error (MAE): $87.82 (Average prediction error)
R-squared (R²): -2.69 (Closer to 1.0 is better)
-------------------------

Sample Test Predictions:


,Actual Amount,Predicted Amount,Difference
0,$85.12,$127.13,-42.01
6,$120.45,$354.84,-234.39
12,$210.50,$347.04,-136.54
1,$12.99,$-11.48,24.47
9,$62.10,$33.85,28.25
17,$98.30,$159.55,-61.25


interactive(children=(Dropdown(description='Select Month:', index=3, layout=Layout(width='300px'), options=('D…

<function __main__.generate_spend_report(selected_month)>

In [4]:
# ----------------------------------------------------------------------
# Interactive Spend Report for Google Colab
# This script uses pandas and plotly to replicate the monthly spend
# visualization within a Python environment (Google Colab).
# ----------------------------------------------------------------------

# 1. Install necessary libraries if not already present in the Colab runtime
# Note: plotly is usually pre-installed in Colab.
# We need scikit-learn for the machine learning model.
!pip install scikit-learn

import pandas as pd
import plotly.express as px
from ipywidgets import interact, Dropdown, Layout
from IPython.display import display, HTML

# ML imports
from sklearn.model_selection import train_test_split
# Using Logistic Regression for classification
from sklearn.linear_model import LogisticRegression
# New imports for text processing and classification metrics/pipeline
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer # Still keeping, though not used in the final pipeline for this task

# --- MOCK TRANSACTION DATA ---
# This mirrors the data structure extracted from your PDF files.
MOCK_TRANSACTIONS = [
    # September 2020 (Aug 14 - Sep 13)
    {'month': 'September 2020', 'date': 'Sep 10', 'description': 'LOBLAWS GROCERY STORE', 'amount': 85.12, 'category': 'Groceries'},
    {'month': 'September 2020', 'date': 'Sep 11', 'description': 'AMAZON PRIME SUBSCRIPTION', 'amount': 12.99, 'category': 'Subscriptions'},
    {'month': 'September 2020', 'date': 'Sep 12', 'description': 'COFFEE SHOP TIM HORTONS', 'amount': 4.50, 'category': 'Food & Dining'},
    {'month': 'September 2020', 'date': 'Sep 13', 'description': 'CIBC PAYMENT - THANK YOU', 'amount': -150.18, 'category': 'Payment'},

    # October 2020 (Sep 14 - Oct 13)
    {'month': 'October 2020', 'date': 'Sep 15', 'description': 'LCBO LIQUOR STORE', 'amount': 35.80, 'category': 'Entertainment'},
    {'month': 'October 2020', 'date': 'Sep 25', 'description': 'BELL CANADA INTERNET', 'amount': 95.00, 'category': 'Utilities'},
    {'month': 'October 2020', 'date': 'Oct 01', 'description': 'WALMART SUPERCENTER', 'amount': 120.45, 'category': 'Shopping'},
    {'month': 'October 2020', 'date': 'Oct 05', 'description': 'PETSMART SUPPLIES', 'amount': 45.99, 'category': 'Pets'},
    {'month': 'October 2020', 'date': 'Oct 10', 'description': 'UBER EATS DELIVERY', 'amount': 32.70, 'category': 'Food & Dining'},
    {'month': 'October 2020', 'date': 'Oct 13', 'description': 'SHELL GAS STATION', 'amount': 62.10, 'category': 'Automotive'},

    # November 2020 (Oct 14 - Nov 13)
    {'month': 'November 2020', 'date': 'Oct 20', 'description': 'METRO GROCERY STORE', 'amount': 155.60, 'category': 'Groceries'},
    {'month': 'November 2020', 'date': 'Oct 25', 'description': 'NETFLIX SUBSCRIPTION', 'amount': 16.99, 'category': 'Subscriptions'},
    {'month': 'November 2020', 'date': 'Nov 01', 'description': 'AMAZON.CA PURCHASE', 'amount': 210.50, 'category': 'Shopping'},
    {'month': 'November 2020', 'date': 'Nov 05', 'description': 'DRY CLEANERS SERVICE', 'amount': 22.00, 'category': 'Personal Care'},
    {'month': 'November 2020', 'date': 'Nov 12', 'description': 'CINEPLEX MOVIES', 'amount': 28.00, 'category': 'Entertainment'},

    # December 2020 (Nov 14 - Dec 13)
    {'month': 'December 2020', 'date': 'Nov 15', 'description': 'ROGERS CELL PHONE BILL', 'amount': 75.00, 'category': 'Utilities'},
    {'month': 'December 2020', 'date': 'Nov 20', 'description': 'APPLE STORE INC PURCHASE', 'amount': 350.99, 'category': 'Shopping'},
    {'month': 'December 2020', 'date': 'Dec 05', 'description': 'FOOD BASICS GROCERY', 'amount': 98.30, 'category': 'Groceries'},
    {'month': 'December 2020', 'date': 'Dec 10', 'description': 'RESTAURANT TAKE OUT', 'amount': 45.00, 'category': 'Food & Dining'},
    {'month': 'December 2020', 'date': 'Dec 12', 'description': 'CAR WASH DETAIL', 'amount': 30.00, 'category': 'Automotive'},
]

# Create a master DataFrame
df_master = pd.DataFrame(MOCK_TRANSACTIONS)
df_master['amount'] = df_master['amount'].round(2) # Ensure amounts are clean
MONTHS = sorted(df_master['month'].unique())
LATEST_MONTH = MONTHS[-1]

# 2. Function to generate the interactive report
def generate_spend_report(selected_month):
    """Generates and displays the category pie chart and transaction table."""
    print(f"--- Spend Report for {selected_month} ---")

    # Filter for the selected month and exclude credits/payments (amount > 0)
    df_monthly_spending = df_master[
        (df_master['month'] == selected_month) &
        (df_master['amount'] > 0) &
        (df_master['category'] != 'Payment')
    ]

    # Aggregate by Category (for Pie Chart)
    if df_monthly_spending.empty:
        print("\nNo purchase transactions found for this period to visualize.")
        return

    df_category_spend = df_monthly_spending.groupby('category')['amount'].sum().reset_index()
    df_category_spend = df_category_spend.sort_values(by='amount', ascending=False)

    # Calculate Summary Stats
    total_spend = df_monthly_spending['amount'].sum()
    num_transactions = df_monthly_spending.shape[0]

    # --- DISPLAY SUMMARY STATS ---
    print(f"\nTotal Purchases: ${total_spend:.2f}")
    print(f"Total Transactions: {num_transactions}")
    print("-" * 30)

    # --- 1. CATEGORY BREAKDOWN (PIE CHART) ---
    print("\n[1] Category Breakdown Pie Chart:")
    fig_pie = px.pie(
        df_category_spend,
        values='amount',
        names='category',
        title=f'Spending Breakdown in {selected_month}',
        color_discrete_sequence=px.colors.qualitative.Bold,
        hole=0.3
    )
    fig_pie.update_traces(textposition='inside', textinfo='percent+label')
    fig_pie.show()

    # --- 2. DETAILED TRANSACTION TABLE ---
    print("\n[2] Detailed Transactions Table:")
    # Only show relevant columns for the table output
    df_table = df_monthly_spending[['date', 'description', 'category', 'amount']]

    # Format the table for better display in Colab
    display(HTML(f"<h3>Detailed Transactions for {selected_month}</h3>"))
    display(df_table.style.set_properties(**{'font-size': '10pt', 'border': '1px solid black'})
                          .format({'amount': '${:.2f}'}))


# 3. Monthly Comparison Chart (Static - does not depend on dropdown)
def plot_monthly_comparison(df):
    """Plots the total monthly spending across all available months (Bar Chart)."""
    # Filter for positive spending (purchases)
    df_purchases = df[df['amount'] > 0]

    # Aggregate total spending per month
    df_monthly_total = df_purchases.groupby('month')['amount'].sum().reset_index()

    print("\n" + "="*50)
    print("Monthly Spending Comparison (Bar Chart)")
    print("="*50)

    fig_bar = px.bar(
        df_monthly_total,
        x='month',
        y='amount',
        title='Total Spending Across All Statement Periods',
        labels={'amount': 'Total Spend ($)', 'month': 'Statement Month'},
        color='amount',
        color_continuous_scale=px.colors.sequential.Sunsetdark
    )
    fig_bar.update_layout(xaxis={'categoryorder':'array', 'categoryarray':MONTHS})
    fig_bar.update_traces(texttemplate='$%{y:.2f}', textposition='outside')
    fig_bar.show()

# 4. Classification Machine Learning Model (Categorize Transaction)
def train_spend_predictor(df):
    """
    Trains a classification model (Logistic Regression) to predict the transaction
    category based on its description text.
    """
    print("\n" + "="*50)
    print("CLASSIFICATION MODEL: PREDICTING TRANSACTION CATEGORY")
    print("="*50)

    # 4.1 Data Preparation: Filter for purchases only
    df_purchases = df[(df['amount'] > 0) & (df['category'] != 'Payment')].copy()

    if df_purchases.shape[0] < 5:
        print("Not enough data points (purchases) to train a meaningful model.")
        return

    # Define features (X) and target (y)
    # X is the transaction description (text feature)
    X = df_purchases['description']
    # y is the transaction category (target class)
    y = df_purchases['category']

    # 4.2 Split Data (70% Train, 30% Test)
    # Removed stratify=y because some categories have only one sample,
    # which is too few for stratification in train_test_split.
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42 # stratify=y removed
    )

    # 4.3 Build Pipeline: Text Vectorization (Feature Engineering) + Classifier
    # Step 1: TfidfVectorizer converts text descriptions into numerical features (vectors).
    # Step 2: LogisticRegression is used as the classifier.
    model_pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(stop_words='english')),
        ('clf', LogisticRegression(max_iter=1000, random_state=42))
    ])

    # 4.4 Train Model
    model_pipeline.fit(X_train, y_train)

    # 4.5 Prediction and Evaluation
    y_pred = model_pipeline.predict(X_test)

    # Evaluate performance
    accuracy = accuracy_score(y_test, y_pred)
    # Set zero_division=0 to handle cases where a category might not appear in the test set
    report = classification_report(y_test, y_pred, zero_division=0, output_dict=False)


    print(f"Model Trained on {X_train.shape[0]} samples, Tested on {X_test.shape[0]} samples.")
    print(f"\n--- Model Performance ---")
    print(f"Classification Accuracy: {accuracy:.2f} (Overall Correct Predictions)")
    print(f"-------------------------\n")
    print("Classification Report (Precision, Recall, F1-Score per Category):")
    print(report)

    # 4.6 Display Sample Predictions
    predictions_df = pd.DataFrame({'Description': X_test, 'Actual Category': y_test, 'Predicted Category': y_pred})

    print("Sample Test Predictions:")
    display(predictions_df.style.set_properties(**{'font-size': '10pt', 'border': '1px solid black'}))


# 5. Execute the interactive elements
# Plot the overall comparison first
plot_monthly_comparison(df_master)

# Train and evaluate the predictive model
train_spend_predictor(df_master)

# Create the Dropdown widget
month_selector = Dropdown(
    options=MONTHS,
    value=LATEST_MONTH,
    description='Select Month:',
    style={'description_width': 'initial'},
    layout=Layout(width='300px')
)

# Use the interact function to link the dropdown to the report generator
display(HTML("<h2><center>Interactive Monthly Spend Analyzer</center></h2>"))
interact(generate_spend_report, selected_month=month_selector)


Monthly Spending Comparison (Bar Chart)



CLASSIFICATION MODEL: PREDICTING TRANSACTION CATEGORY
Model Trained on 13 samples, Tested on 6 samples.

--- Model Performance ---
Classification Accuracy: 0.00 (Overall Correct Predictions)
-------------------------

Classification Report (Precision, Recall, F1-Score per Category):
               precision    recall  f1-score   support

   Automotive       0.00      0.00      0.00       1.0
Food & Dining       0.00      0.00      0.00       0.0
    Groceries       0.00      0.00      0.00       2.0
     Shopping       0.00      0.00      0.00       2.0
Subscriptions       0.00      0.00      0.00       1.0

     accuracy                           0.00       6.0
    macro avg       0.00      0.00      0.00       6.0
 weighted avg       0.00      0.00      0.00       6.0

Sample Test Predictions:


,Description,Actual Category,Predicted Category
0,LOBLAWS GROCERY STORE,Groceries,Food & Dining
6,WALMART SUPERCENTER,Shopping,Food & Dining
12,AMAZON.CA PURCHASE,Shopping,Food & Dining
1,AMAZON PRIME SUBSCRIPTION,Subscriptions,Food & Dining
9,SHELL GAS STATION,Automotive,Food & Dining
17,FOOD BASICS GROCERY,Groceries,Food & Dining


interactive(children=(Dropdown(description='Select Month:', index=3, layout=Layout(width='300px'), options=('D…

<function __main__.generate_spend_report(selected_month)>

In [5]:
# ----------------------------------------------------------------------
# Interactive Spend Report for Google Colab
# This script uses pandas and plotly to replicate the monthly spend
# visualization within a Python environment (Google Colab).
# ----------------------------------------------------------------------

# 1. Install necessary libraries if not already present in the Colab runtime
# Note: plotly is usually pre-installed in Colab.
# We need scikit-learn for the machine learning model.
!pip install scikit-learn

import pandas as pd
import plotly.express as px
from ipywidgets import interact, Dropdown, Layout
from IPython.display import display, HTML
import numpy as np # Added for array handling

# ML imports
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler # Added StandardScaler
from sklearn.compose import ColumnTransformer
from scipy.sparse import hstack # Added for combining sparse and dense feature matrices

# --- MOCK TRANSACTION DATA ---
# This mirrors the data structure extracted from your PDF files.
MOCK_TRANSACTIONS = [
    # September 2020 (Aug 14 - Sep 13)
    {'month': 'September 2020', 'date': 'Sep 10', 'description': 'LOBLAWS GROCERY STORE', 'amount': 85.12, 'category': 'Groceries'},
    {'month': 'September 2020', 'date': 'Sep 11', 'description': 'AMAZON PRIME SUBSCRIPTION', 'amount': 12.99, 'category': 'Subscriptions'},
    {'month': 'September 2020', 'date': 'Sep 12', 'description': 'COFFEE SHOP TIM HORTONS', 'amount': 4.50, 'category': 'Food & Dining'},
    {'month': 'September 2020', 'date': 'Sep 13', 'description': 'CIBC PAYMENT - THANK YOU', 'amount': -150.18, 'category': 'Payment'},

    # October 2020 (Sep 14 - Oct 13)
    {'month': 'October 2020', 'date': 'Sep 15', 'description': 'LCBO LIQUOR STORE', 'amount': 35.80, 'category': 'Entertainment'},
    {'month': 'October 2020', 'date': 'Sep 25', 'description': 'BELL CANADA INTERNET', 'amount': 95.00, 'category': 'Utilities'},
    {'month': 'October 2020', 'date': 'Oct 01', 'description': 'WALMART SUPERCENTER', 'amount': 120.45, 'category': 'Shopping'},
    {'month': 'October 2020', 'date': 'Oct 05', 'description': 'PETSMART SUPPLIES', 'amount': 45.99, 'category': 'Pets'},
    {'month': 'October 2020', 'date': 'Oct 10', 'description': 'UBER EATS DELIVERY', 'amount': 32.70, 'category': 'Food & Dining'},
    {'month': 'October 2020', 'date': 'Oct 13', 'description': 'SHELL GAS STATION', 'amount': 62.10, 'category': 'Automotive'},

    # November 2020 (Oct 14 - Nov 13)
    {'month': 'November 2020', 'date': 'Oct 20', 'description': 'METRO GROCERY STORE', 'amount': 155.60, 'category': 'Groceries'},
    {'month': 'November 2020', 'date': 'Oct 25', 'description': 'NETFLIX SUBSCRIPTION', 'amount': 16.99, 'category': 'Subscriptions'},
    {'month': 'November 2020', 'date': 'Nov 01', 'description': 'AMAZON.CA PURCHASE', 'amount': 210.50, 'category': 'Shopping'},
    {'month': 'November 2020', 'date': 'Nov 05', 'description': 'DRY CLEANERS SERVICE', 'amount': 22.00, 'category': 'Personal Care'},
    {'month': 'November 2020', 'date': 'Nov 12', 'description': 'CINEPLEX MOVIES', 'amount': 28.00, 'category': 'Entertainment'},

    # December 2020 (Nov 14 - Dec 13)
    {'month': 'December 2020', 'date': 'Nov 15', 'description': 'ROGERS CELL PHONE BILL', 'amount': 75.00, 'category': 'Utilities'},
    {'month': 'December 2020', 'date': 'Nov 20', 'description': 'APPLE STORE INC PURCHASE', 'amount': 350.99, 'category': 'Shopping'},
    {'month': 'December 2020', 'date': 'Dec 05', 'description': 'FOOD BASICS GROCERY', 'amount': 98.30, 'category': 'Groceries'},
    {'month': 'December 2020', 'date': 'Dec 10', 'description': 'RESTAURANT TAKE OUT', 'amount': 45.00, 'category': 'Food & Dining'},
    {'month': 'December 2020', 'date': 'Dec 12', 'description': 'CAR WASH DETAIL', 'amount': 30.00, 'category': 'Automotive'},
]

# Create a master DataFrame
df_master = pd.DataFrame(MOCK_TRANSACTIONS)
df_master['amount'] = df_master['amount'].round(2) # Ensure amounts are clean
MONTHS = sorted(df_master['month'].unique())
LATEST_MONTH = MONTHS[-1]

# 2. Function to generate the interactive report
def generate_spend_report(selected_month):
    """Generates and displays the category pie chart and transaction table."""
    print(f"--- Spend Report for {selected_month} ---")

    # Filter for the selected month and exclude credits/payments (amount > 0)
    df_monthly_spending = df_master[
        (df_master['month'] == selected_month) &
        (df_master['amount'] > 0) &
        (df_master['category'] != 'Payment')
    ]

    # Aggregate by Category (for Pie Chart)
    if df_monthly_spending.empty:
        print("\nNo purchase transactions found for this period to visualize.")
        return

    df_category_spend = df_monthly_spending.groupby('category')['amount'].sum().reset_index()
    df_category_spend = df_category_spend.sort_values(by='amount', ascending=False)

    # Calculate Summary Stats
    total_spend = df_monthly_spending['amount'].sum()
    num_transactions = df_monthly_spending.shape[0]

    # --- DISPLAY SUMMARY STATS ---
    print(f"\nTotal Purchases: ${total_spend:.2f}")
    print(f"Total Transactions: {num_transactions}")
    print("-" * 30)

    # --- 1. CATEGORY BREAKDOWN (PIE CHART) ---
    print("\n[1] Category Breakdown Pie Chart:")
    fig_pie = px.pie(
        df_category_spend,
        values='amount',
        names='category',
        title=f'Spending Breakdown in {selected_month}',
        color_discrete_sequence=px.colors.qualitative.Bold,
        hole=0.3
    )
    fig_pie.update_traces(textposition='inside', textinfo='percent+label')
    fig_pie.show()

    # --- 2. DETAILED TRANSACTION TABLE ---
    print("\n[2] Detailed Transactions Table:")
    # Only show relevant columns for the table output
    df_table = df_monthly_spending[['date', 'description', 'category', 'amount']]

    # Format the table for better display in Colab
    display(HTML(f"<h3>Detailed Transactions for {selected_month}</h3>"))
    display(df_table.style.set_properties(**{'font-size': '10pt', 'border': '1px solid black'})
                          .format({'amount': '${:.2f}'}))


# 3. Monthly Comparison Chart (Static - does not depend on dropdown)
def plot_monthly_comparison(df):
    """Plots the total monthly spending across all available months (Bar Chart)."""
    # Filter for positive spending (purchases)
    df_purchases = df[df['amount'] > 0]

    # Aggregate total spending per month
    df_monthly_total = df_purchases.groupby('month')['amount'].sum().reset_index()

    print("\n" + "="*50)
    print("Monthly Spending Comparison (Bar Chart)")
    print("="*50)

    fig_bar = px.bar(
        df_monthly_total,
        x='month',
        y='amount',
        title='Total Spending Across All Statement Periods',
        labels={'amount': 'Total Spend ($)', 'month': 'Statement Month'},
        color='amount',
        color_continuous_scale=px.colors.sequential.Sunsetdark
    )
    fig_bar.update_layout(xaxis={'categoryorder':'array', 'categoryarray':MONTHS})
    fig_bar.update_traces(texttemplate='$%{y:.2f}', textposition='outside')
    fig_bar.show()

# 4. Classification Machine Learning Model (Categorize Transaction)
def train_spend_predictor(df):
    """
    Trains a classification model (Logistic Regression) to predict the transaction
    category using all features: description (text), month (categorical),
    day (numerical), and amount (numerical).
    """
    print("\n" + "="*50)
    print("CLASSIFICATION MODEL: PREDICTING TRANSACTION CATEGORY (ALL FEATURES)")
    print("="*50)

    # 4.1 Data Preparation: Filter for purchases and create 'day' feature
    df_purchases = df[(df['amount'] > 0) & (df['category'] != 'Payment')].copy()

    # Feature Engineering: Extract day of the month
    # Assumes date format like 'Sep 10'
    df_purchases['day'] = df_purchases['date'].apply(lambda x: int(x.split(' ')[1]))

    if df_purchases.shape[0] < 5:
        print("Not enough data points (purchases) to train a meaningful model.")
        return

    # Define features (X) and target (y)
    X = df_purchases[['description', 'month', 'day', 'amount']]
    y = df_purchases['category']

    # 4.2 Split Data (70% Train, 30% Test)
    # Removed stratify=y because some categories have only one sample, which is too few for stratification.
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42 # stratify=y removed
    )

    # 4.3 Define Preprocessors for different feature types

    # 1. Text Pipeline (TF-IDF Vectorization)
    tfidf = TfidfVectorizer(stop_words='english')

    # 2. Non-Text Preprocessor (One-Hot Encoding, Scaling)
    # Scales numerical features to prevent 'amount' from dominating
    non_text_preprocessor = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(handle_unknown='ignore'), ['month']),
            ('num', StandardScaler(), ['day', 'amount'])
        ],
        remainder='passthrough'
    )

    # 4.4 Feature Transformation and Combination (Avoiding Pipeline complexity)

    # Transform Text Features
    X_train_text = tfidf.fit_transform(X_train['description'])
    X_test_text = tfidf.transform(X_test['description'])

    # Transform Non-Text Features
    X_train_other = non_text_preprocessor.fit_transform(X_train[['month', 'day', 'amount']])
    X_test_other = non_text_preprocessor.transform(X_test[['month', 'day', 'amount']])

    # Combine Text (Sparse) and Other (Dense) Features
    X_train_combined = hstack([X_train_text, X_train_other])
    X_test_combined = hstack([X_test_text, X_test_other])

    # 4.5 Train Model (Logistic Regression handles sparse/high-dimensional data well)
    model = LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')
    model.fit(X_train_combined, y_train)

    # 4.6 Prediction and Evaluation
    y_pred = model.predict(X_test_combined)

    # Evaluate performance
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, zero_division=0, output_dict=False)


    print(f"Model Trained on {X_train.shape[0]} samples, Tested on {X_test.shape[0]} samples.")
    print(f"\n--- Model Performance (Using ALL Features) ---")
    print(f"Classification Accuracy: {accuracy:.2f} (Overall Correct Predictions)")
    print(f"---------------------------------------------\n")
    print("Classification Report (Precision, Recall, F1-Score per Category):")
    print(report)

    # 4.7 Display Sample Predictions
    predictions_df = X_test.copy()
    predictions_df['Actual Category'] = y_test
    predictions_df['Predicted Category'] = y_pred

    print("Sample Test Predictions:")
    display(predictions_df.style.set_properties(**{'font-size': '10pt', 'border': '1px solid black'}))


# 5. Execute the interactive elements
# Plot the overall comparison first
plot_monthly_comparison(df_master)

# Train and evaluate the predictive model
train_spend_predictor(df_master)

# Create the Dropdown widget
month_selector = Dropdown(
    options=MONTHS,
    value=LATEST_MONTH,
    description='Select Month:',
    style={'description_width': 'initial'},
    layout=Layout(width='300px')
)

# Use the interact function to link the dropdown to the report generator
display(HTML("<h2><center>Interactive Monthly Spend Analyzer</center></h2>"))
interact(generate_spend_report, selected_month=month_selector)


Monthly Spending Comparison (Bar Chart)



CLASSIFICATION MODEL: PREDICTING TRANSACTION CATEGORY (ALL FEATURES)
Model Trained on 13 samples, Tested on 6 samples.

--- Model Performance (Using ALL Features) ---
Classification Accuracy: 0.00 (Overall Correct Predictions)
---------------------------------------------

Classification Report (Precision, Recall, F1-Score per Category):
               precision    recall  f1-score   support

   Automotive       0.00      0.00      0.00       1.0
Food & Dining       0.00      0.00      0.00       0.0
    Groceries       0.00      0.00      0.00       2.0
Personal Care       0.00      0.00      0.00       0.0
         Pets       0.00      0.00      0.00       0.0
     Shopping       0.00      0.00      0.00       2.0
Subscriptions       0.00      0.00      0.00       1.0

     accuracy                           0.00       6.0
    macro avg       0.00      0.00      0.00       6.0
 weighted avg       0.00      0.00      0.00       6.0

Sample Test Predictions:


,description,month,day,amount,Actual Category,Predicted Category
0,LOBLAWS GROCERY STORE,September 2020,10,85.120000,Groceries,Food & Dining
6,WALMART SUPERCENTER,October 2020,1,120.450000,Shopping,Pets
12,AMAZON.CA PURCHASE,November 2020,1,210.500000,Shopping,Personal Care
1,AMAZON PRIME SUBSCRIPTION,September 2020,11,12.990000,Subscriptions,Food & Dining
9,SHELL GAS STATION,October 2020,13,62.100000,Automotive,Food & Dining
17,FOOD BASICS GROCERY,December 2020,5,98.300000,Groceries,Food & Dining


interactive(children=(Dropdown(description='Select Month:', index=3, layout=Layout(width='300px'), options=('D…

<function __main__.generate_spend_report(selected_month)>